# Tensor processing notebook

## Imports

In [17]:
import os
from datetime import date

try:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import tensorflow as tf
    import keras

except:
    os.system("pip install numpy")
    os.system("pip install pandas")
    os.system("pip install matplotlib")
    os.system("pip install seaborn")
    os.system("pip install tensorflow")
    os.system("pip install keras")

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import tensorflow as tf
    import keras


## Main code

In [28]:

# Your CSV dataset
datasets = ['sionna_madrid', 'sionna_madrid_concrete_ground']

for dataset in datasets:
    df = pd.read_csv(f'{dataset}/scene_charact_{dataset}2025-05-15.csv', sep=',')
    
    # Define bin edges and labels (in dBm)
    bins = [float('-inf'), -120, -100, -90, -80, float('inf')]
    labels = ['very_poor', 'poor', 'fair', 'good', 'excellent']
    
    # Add new columns if not already present
    for label in labels:
        if f'percent_{label}' not in df.columns:
            df[f'percent_{label}'] = np.nan
    
    for idx, row in df.iterrows():
        name = str(row['scene_name'])
        # Change to the actual path where your tensors are stored
        tensor_path = os.path.join(f'{dataset}', name, 'cm_tensor_dB.npy')
        if not os.path.exists(tensor_path):
            print(f"Tensor not found for scene {name}")
            continue
    
        # Load tensor (shape: 3,1,200,200)
        tensor = np.load(tensor_path)
        # Flatten all values for global statistics and binning
        # values = tensor.flatten()

        print(type(tensor))
        tensor_tf = tf.convert_to_tensor(tensor)
        tensor_tf_max = tf.reduce_max(tensor_tf, axis=0)
        print(f'Esta es la dimension: {tf.shape(tensor_tf_max)}')
        print(f"Este es el tensor: {tensor_tf_max}")

        # Histogram for RSRP classes
        values = tensor_tf_max
        # total_cells = values.size
        total_cells = tf.size(values)
        print(f"Este es el size: {total_cells}")
        hist, _ = np.histogram(values, bins=bins)
        pct = hist / total_cells

        # Tensor statistics
        tensor_tf = tf.convert_to_tensor(values)
        tensor_max = float(tf.reduce_max(tensor_tf).numpy())
        tensor_min = float(tf.reduce_min(tensor_tf).numpy())
        tensor_mean = float(tf.reduce_mean(tensor_tf).numpy())
        tensor_median = float(np.median(values))

        # Assign stats to DataFrame
        df.at[idx, 'tensor_max [dBm]'] = tensor_max
        df.at[idx, 'tensor_min [dBm]'] = tensor_min
        df.at[idx, 'tensor_mean [dBm]'] = tensor_mean
        df.at[idx, 'tensor_median [dBm]'] = tensor_median
        for l, v in zip(labels, pct):
            df.at[idx, f'percent_{l}'] = v
    
    # Save the enriched CSV
    df.to_csv(f'{dataset}/scene_charact_{dataset}_TENSOR_info_{date.today()}.csv', sep=',', index=False)
    print("CSV updated with RSS/RSRP classification percentages.")


CSV updated and saved as sionna_madrid/scene_charact_sionna_madrid_TENSOR_info_2025-05-22.csv
CSV updated and saved as sionna_madrid_concrete_ground/scene_charact_sionna_madrid_concrete_ground_TENSOR_info_2025-05-22.csv


In [15]:
dataset = 'sionna_madrid'
df = pd.read_csv(f'{dataset}/scene_charact_{dataset}_TENSOR_info_2025-05-15.csv', sep=',')
df.head

# List of columns to sum
cols_to_sum = ['percent_very_poor', 'percent_poor', 'percent_fair','percent_good','percent_excellent']

# Create the new column with the sum for each row
df['sum_selected_percents'] = df[cols_to_sum].sum(axis=1)

# Show the result
print(df[['percent_very_poor', 'percent_poor', 'percent_fair','percent_good','percent_excellent','sum_selected_percents']])

# let us now get the max, min, mean and median values for the tensor 

     percent_very_poor  percent_poor  percent_fair  percent_good  \
0             0.626483      0.243558      0.072117      0.025450   
1             0.636808      0.202992      0.082750      0.044567   
2             0.459008      0.289558      0.112642      0.061475   
3             0.327160      0.408687      0.140806      0.052479   
4             0.367592      0.334767      0.181342      0.045558   
..                 ...           ...           ...           ...   
651           0.584123      0.201348      0.094500      0.048000   
652           0.431992      0.271976      0.146415      0.101016   
653           0.665383      0.153992      0.063158      0.052958   
654           0.452173      0.222787      0.149085      0.084009   
655           0.666425      0.196508      0.066217      0.034883   

     percent_excellent  sum_selected_percents  
0             0.032392                    1.0  
1             0.032883                    1.0  
2             0.077317                 

In [19]:
tensor = tf.constant([[9, 8, 7],[1, 9, 10]])
result = tf.reduce_max(tensor, axis=0)
print(result)

tf.Tensor([ 9  9 10], shape=(3,), dtype=int32)
